In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # able to run multiple command in 1 line


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
folder = '/Users/amarindraardinova/Desktop/DataScience/Coursera/Project/Analize Data in Model Car Database/'
file_path = folder + 'join_table.csv' 

In [3]:
print(os.path.exists(file_path))

True


In [4]:
df = pd.read_csv(file_path,sep=';')


# EDA

In [5]:
#df = df[sorted(df.columns)]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12015 entries, 0 to 12014
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerNumber          12015 non-null  int64  
 1   customerName            12015 non-null  object 
 2   contactLastName         12015 non-null  object 
 3   contactFirstName        12015 non-null  object 
 4   phone                   12015 non-null  object 
 5   addressLine1            12015 non-null  object 
 6   addressLine2            1827 non-null   object 
 7   city                    12015 non-null  object 
 8   state                   4750 non-null   object 
 9   postalCode              11545 non-null  object 
 10  country                 12015 non-null  object 
 11  salesRepEmployeeNumber  12015 non-null  int64  
 12  creditLimit             12015 non-null  float64
 13  orderNumber             12015 non-null  int64  
 14  orderDate               12015 non-null

## preprocessing : cleaning, rename, duplicate, missing values, feature engineering

In [7]:
# cleaning duplicates features :

columns_to_keep = [col for col in df.columns if not col.endswith('.1')]
df = df [columns_to_keep]

columns_to_keep = [col for col in df.columns if not col.endswith('.2')]
df = df [columns_to_keep]

df.drop(['htmlDescription','image'], axis=1, inplace=True)

In [9]:
df.duplicated().any()

np.False_

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12015 entries, 0 to 12014
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerNumber          12015 non-null  int64  
 1   customerName            12015 non-null  object 
 2   contactLastName         12015 non-null  object 
 3   contactFirstName        12015 non-null  object 
 4   phone                   12015 non-null  object 
 5   addressLine1            12015 non-null  object 
 6   addressLine2            1827 non-null   object 
 7   city                    12015 non-null  object 
 8   state                   4750 non-null   object 
 9   postalCode              11545 non-null  object 
 10  country                 12015 non-null  object 
 11  salesRepEmployeeNumber  12015 non-null  int64  
 12  creditLimit             12015 non-null  float64
 13  orderNumber             12015 non-null  int64  
 14  orderDate               12015 non-null

## Lost & Profit

In [10]:
product_eda = df[['warehouseName', 'status', 'productLine', 'productName',
    'quantityInStock', 'quantityOrdered', 'buyPrice', 'priceEach', 'MSRP']].copy()

product_eda['TotalPurchaseValue'] = product_eda['buyPrice'] * product_eda['quantityInStock']
product_eda['TotalSalesValue']    = product_eda['priceEach'] * product_eda['quantityOrdered']
product_eda['LostProfit']         = product_eda['TotalSalesValue'] - product_eda['TotalPurchaseValue']

product_eda.sample()

,warehouseName,status,productLine,productName,quantityInStock,quantityOrdered,buyPrice,priceEach,MSRP,TotalPurchaseValue,TotalSalesValue,LostProfit
6049,North,Shipped,Motorcycles,2003 Harley-Davidson Eagle Drag Bike,5582,36,91.02,154.93,193.66,508073.64,5577.48,-502496.16


In [11]:
### lost and profit
total_purchase      = product_eda['TotalPurchaseValue'].sum()
total_sells         = product_eda['TotalSalesValue'].sum()
financial_condition = product_eda['LostProfit'].sum()

formatted_total_purchase      = f"{total_purchase:,.2f}" 
formatted_total_sells         = f"{total_sells:,.2f}" 
formatted_financial_condition = f"{financial_condition:,.2f}" 
finstatus = "Lost" if financial_condition < 0 else "Profit"

print(f"Total Purchase      :  {formatted_total_purchase}")
print(f"Total Sells         :     {formatted_total_sells}")
print(f"Lost & Profit       : {formatted_financial_condition}")
print(f"Financial Status    :         {finstatus}")

Total Purchase      :  3,348,304,254.82
Total Sells         :     38,570,229.99
Lost & Profit       : -3,309,734,024.83
Financial Status    :         Lost


## product unique intersect of mySQL and python

In [12]:
productNameUnique_df = pd.Series(sorted(df.productName.unique().tolist())).to_frame(name='productName_df')
productNameUnique_df.sample()

,productName_df
0,18th Century Vintage Horse Carriage


In [13]:
productName_distinct = folder + 'productName_distinct.csv'
productName_distinct_sql = pd.read_csv(productName_distinct, sep=';')
productName_distinct_sql.sample()

,productName_mysql
79,1993 Mazda RX-7


In [14]:
productName_distinct_sql[~productName_distinct_sql['productName_mysql'].isin(productNameUnique_df['productName_df'])]

,productName_mysql
76,1985 Toyota Supra


In [15]:
columns_to_keep = [col for col in df.columns if not col.endswith('.1')]
df = df [columns_to_keep]

columns_to_keep = [col for col in df.columns if not col.endswith('.2')]
df = df [columns_to_keep]

### groupby warehouse

In [16]:
df.columns

Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'salesRepEmployeeNumber', 'creditLimit', 'orderNumber',
       'orderDate', 'requiredDate', 'shippedDate', 'status', 'comments',
       'productCode', 'quantityOrdered', 'priceEach', 'orderLineNumber',
       'productName', 'productLine', 'productScale', 'productVendor',
       'productDescription', 'quantityInStock', 'warehouseCode', 'buyPrice',
       'MSRP', 'textDescription', 'warehouseName', 'warehousePctCap',
       'checkNumber', 'paymentDate', 'amount', 'employeeNumber', 'lastName',
       'firstName', 'extension', 'email', 'officeCode', 'reportsTo',
       'jobTitle', 'territory'],
      dtype='object')

In [17]:
warehouse_cnt = df.groupby('warehouseName')[['productName', 'quantityInStock','quantityOrdered']].agg({'productName': ['count', 'nunique'], 'quantityInStock': 'sum', 'quantityOrdered': 'sum'}).reset_index()
warehouse_cnt.columns = ['warehouseName', 'totalrows', 'unique_product', 'TotalInStock','TotalOrdered']
warehouse_cnt

,warehouseName,totalrows,unique_product,TotalInStock,TotalOrdered
0,East,4341,37,24863942,153987
1,North,2363,25,12588144,84561
2,South,2761,23,9675107,97967
3,West,2550,24,13434712,88430


In [18]:
### Eda Sales on East 
warehouse_east = df[df['warehouseName'] == 'East']
warehouse_east.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory
8315,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.0,10347,2004-11-29,2004-12-07,2004-11-30,Shipped,Can we deliver the new Ford Mustang models by ...,S10_4962,27,132.97,2,1962 LanciaA Delta 16V,Classic Cars,1:10,Second Gear Diecast,Features include: Turnable front wheels; steer...,6791,b,103.42,147.74,Attention car enthusiasts: Make your wildest c...,East,67,NR27552,2004-03-10,44894.74,1611,Fixter,Andy,x101,afixter@classicmodelcars.com,6,1088,Sales Rep,APAC


### grouby status

In [19]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory
3356,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216,114200.0,10192,2003-11-20,2003-11-29,2003-11-25,Shipped,NaN,S12_4675,27,99.04,16,1969 Dodge Charger,Classic Cars,1:12,Welly Diecast Productions,Detailed model of the 1969 Dodge Charger. This...,7323,b,58.73,115.16,Attention car enthusiasts: Make your wildest c...,East,67,PN238558,2003-12-05,55425.77,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,NaN


In [20]:
warehouse_status = df.groupby(['warehouseName','status'])[['quantityInStock']].sum().reset_index()
warehouse_status

,warehouseName,status,quantityInStock
0,East,Cancelled,266884
1,East,Disputed,90948
2,East,In Process,355603
3,East,On Hold,84340
4,East,Resolved,155673
5,East,Shipped,23910494
6,North,Cancelled,764040
7,North,Disputed,561059
8,North,On Hold,92672
9,North,Resolved,666708


In [21]:
warehouse_status.to_excel('warehouse_status.xlsx', sheet_name='warehouse_status', index=False) 

### Warehouse East : Count Values

In [22]:
warehouseEast_df = df[df['warehouseName'] == 'East']
warehouseEast_df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory
3576,357,GiftsForHim.com,MacKinlay,Wales,64-9-3763555,199 Great North Road,NaN,Auckland,NaN,NaN,New Zealand,1612,77700.0,10202,2003-12-02,2003-12-09,2003-12-06,Shipped,NaN,S24_4620,50,75.18,1,1961 Chevrolet Impala,Classic Cars,1:18,Classic Metal Creations,This 1:18 scale precision die-cast reproductio...,7869,b,32.33,80.84,Attention car enthusiasts: Make your wildest c...,East,67,NB291497,2004-05-15,36442.34,1612,Marsh,Peter,x102,pmarsh@classicmodelcars.com,6,1088,Sales Rep,APAC


In [23]:
warehouseEast_df.quantityInStock.sum()

np.int64(24863942)

In [24]:
warehouseEast_df_cv = warehouseEast_df['productName'].value_counts().reset_index()
warehouseEast_df_cv.sample()
warehouseEast_df_cv['count'].sum()

,productName,count
14,1971 Alpine Renault 1600s,118


np.int64(4341)

In [25]:
warehouseEast_df_cv.sample()

,productName,count
31,1962 LanciaA Delta 16V,93


In [26]:
wh_east_sql =  folder + 'warehouseEast.csv'
warehouse_sql = pd.read_csv(wh_east_sql, sep=';')

In [27]:
warehouse_sql.shape[0]

110

In [28]:
warehouse_sql.sample()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,warehouseCode.1,warehouseName,warehousePctCap
9,S24_1785,1928 British Royal Navy Airplane,Planes,1:24,Classic Metal Creations,Official logos and insignias,3627,a,66.74,109.42,a,North,72


In [29]:
warehouse_sql.quantityInStock.min()

np.int64(15)